In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('../data/raw_data/cc-est2017-alldata.csv', encoding = "ISO-8859-1")  #Age, sex, gender
# Each year, the United States Census Bureau produces and publishes estimates of the population for the nation,
# states, counties, state/county equivalents, and Puerto Rico

In [3]:
#TO MAKE A FIPS FILE WITH COUNT NAMES, EXPORTED THE FOLLOWING

# unique_cty_states = df[['CTYNAME', 'STNAME']].copy()
# unique_cty_states = unique_cty_states.drop_duplicates()
# unique_cty_states.to_csv('../data/Census-County-names')

In [4]:
df.index

RangeIndex(start=0, stop=596980, step=1)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 596980 entries, 0 to 596979
Data columns (total 80 columns):
SUMLEV          596980 non-null int64
STATE           596980 non-null int64
COUNTY          596980 non-null int64
STNAME          596980 non-null object
CTYNAME         596980 non-null object
YEAR            596980 non-null int64
AGEGRP          596980 non-null int64
TOT_POP         596980 non-null int64
TOT_MALE        596980 non-null int64
TOT_FEMALE      596980 non-null int64
WA_MALE         596980 non-null int64
WA_FEMALE       596980 non-null int64
BA_MALE         596980 non-null int64
BA_FEMALE       596980 non-null int64
IA_MALE         596980 non-null int64
IA_FEMALE       596980 non-null int64
AA_MALE         596980 non-null int64
AA_FEMALE       596980 non-null int64
NA_MALE         596980 non-null int64
NA_FEMALE       596980 non-null int64
TOM_MALE        596980 non-null int64
TOM_FEMALE      596980 non-null int64
WAC_MALE        596980 non-null int64
WAC_FEMALE   

# Step 1: Add FIPS columns

In [6]:
fips = pd.read_csv('../data/county_fips_mapping.csv')

In [7]:
fips.head()

,Unnamed: 0,fips,county,state_abbr,state,census_CTYNAME
0,0,1001,Autauga,AL,Alabama,Autauga County
1,1,1003,Baldwin,AL,Alabama,Baldwin County
2,2,1005,Barbour,AL,Alabama,Barbour County
3,3,1007,Bibb,AL,Alabama,Bibb County
4,4,1009,Blount,AL,Alabama,Blount County


In [8]:
fips.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3236 entries, 0 to 3235
Data columns (total 6 columns):
Unnamed: 0        3236 non-null int64
fips              3236 non-null int64
county            3236 non-null object
state_abbr        3236 non-null object
state             3236 non-null object
census_CTYNAME    3236 non-null object
dtypes: int64(2), object(4)
memory usage: 151.8+ KB


In [9]:
fips['StateCounty'] = fips['state'] + fips['census_CTYNAME']
df['StateCounty'] = df["STNAME"] + df["CTYNAME"]
df.shape

(596980, 81)

In [10]:
newdf = df.merge(fips[['StateCounty', "fips", "state", "county", 'census_CTYNAME']], on = "StateCounty", how = 'left')
newdf.head()

,SUMLEV,STATE,COUNTY,STNAME,CTYNAME,YEAR,AGEGRP,TOT_POP,TOT_MALE,TOT_FEMALE,...,HIAC_FEMALE,HAAC_MALE,HAAC_FEMALE,HNAC_MALE,HNAC_FEMALE,StateCounty,fips,state,county,census_CTYNAME
0,50,1,1,Alabama,Autauga County,1,0,54571,26569,28002,...,32,9,11,19,10,AlabamaAutauga County,1001.0,Alabama,Autauga,Autauga County
1,50,1,1,Alabama,Autauga County,1,1,3579,1866,1713,...,1,0,0,2,1,AlabamaAutauga County,1001.0,Alabama,Autauga,Autauga County
2,50,1,1,Alabama,Autauga County,1,2,3991,2001,1990,...,7,2,3,2,0,AlabamaAutauga County,1001.0,Alabama,Autauga,Autauga County
3,50,1,1,Alabama,Autauga County,1,3,4290,2171,2119,...,5,2,1,1,1,AlabamaAutauga County,1001.0,Alabama,Autauga,Autauga County
4,50,1,1,Alabama,Autauga County,1,4,4290,2213,2077,...,2,2,1,3,1,AlabamaAutauga County,1001.0,Alabama,Autauga,Autauga County


In [11]:
newdf.shape

(596980, 85)

## Step 2: Subset for Year = 1 (2010 Census) 

In [21]:
newdf = newdf[newdf['YEAR'] == 1]

## Step 3: Creating % Male population

In [22]:
male_df = newdf[['STNAME', 'CTYNAME', "fips", "YEAR", "AGEGRP", "TOT_POP", "TOT_MALE", "TOT_FEMALE", ]]

In [23]:
male_df.head()

,STNAME,CTYNAME,fips,YEAR,AGEGRP,TOT_POP,TOT_MALE,TOT_FEMALE
0,Alabama,Autauga County,1001.0,1,0,54571,26569,28002
1,Alabama,Autauga County,1001.0,1,1,3579,1866,1713
2,Alabama,Autauga County,1001.0,1,2,3991,2001,1990
3,Alabama,Autauga County,1001.0,1,3,4290,2171,2119
4,Alabama,Autauga County,1001.0,1,4,4290,2213,2077


In [24]:
male_df["pct_male"] =  (male_df["TOT_MALE"] / male_df["TOT_POP"])*100

/Users/LB/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [25]:
male_df.head()

,STNAME,CTYNAME,fips,YEAR,AGEGRP,TOT_POP,TOT_MALE,TOT_FEMALE,pct_male
0,Alabama,Autauga County,1001.0,1,0,54571,26569,28002,48.687032
1,Alabama,Autauga County,1001.0,1,1,3579,1866,1713,52.137469
2,Alabama,Autauga County,1001.0,1,2,3991,2001,1990,50.137810
3,Alabama,Autauga County,1001.0,1,3,4290,2171,2119,50.606061
4,Alabama,Autauga County,1001.0,1,4,4290,2213,2077,51.585082


## Step 3: Creating Age Dummies (Using Year = 1 =  2010 Census Population Numbers)

In [17]:
# The key for AGEGRP is as follows:
# 0 = Total
# 1 = Age 0 to 4 years
# 2 = Age 5 to 9 years
# 3 = Age 10 to 14 years
# 4 = Age 15 to 19 years
# 5 = Age 20 to 24 years
# 6 = Age 25 to 29 years
# 7 = Age 30 to 34 years
# 8 = Age 35 to 39 years
# 9 = Age 40 to 44 years
# 10 = Age 45 to 49 years
# 11 = Age 50 to 54 years
# 12 = Age 55 to 59 years
# 13 = Age 60 to 64 years
# 14 = Age 65 to 69 years
# 15 = Age 70 to 74 years
# 16 = Age 75 to 79 years
# 17 = Age 80 to 84 years
# 18 = Age 85 years or older

In [26]:
male_df.head()

,STNAME,CTYNAME,fips,YEAR,AGEGRP,TOT_POP,TOT_MALE,TOT_FEMALE,pct_male
0,Alabama,Autauga County,1001.0,1,0,54571,26569,28002,48.687032
1,Alabama,Autauga County,1001.0,1,1,3579,1866,1713,52.137469
2,Alabama,Autauga County,1001.0,1,2,3991,2001,1990,50.137810
3,Alabama,Autauga County,1001.0,1,3,4290,2171,2119,50.606061
4,Alabama,Autauga County,1001.0,1,4,4290,2213,2077,51.585082


In [27]:
Age_Mapping = {
    1: "2010:AgeGroup01:0-4",
    2: "2010:AgeGroup02:5-9",
    3: "2010:AgeGroup03:10-14",
    4: "2010:AgeGroup04:15-19",
    5: "2010:AgeGroup05:20-24",
    6: "2010:AgeGroup06:25-29",
    7: "2010:AgeGroup07:30-34",
    8: "2010:AgeGroup08:35-39",
    9: "2010:AgeGroup09:40-44",
    10: "2010:AgeGroup10:45-49",
    11: "2010:AgeGroup11:50-54",
    12: "2010:AgeGroup12:55-59",
    13: "2010:AgeGroup13:60-64",
    14: "2010:AgeGroup14:65-69",
    15: "2010:AgeGroup15:70-74",
    16: "2010:AgeGroup16:75-79",
    17: "2010:AgeGroup17:80-84",
    18: "2010:AgeGroup18:85+",
}

In [29]:
age_df = male_df[male_df["AGEGRP"] == 0][["STNAME", "CTYNAME", "fips", "AGEGRP", "TOT_POP", "pct_male" ]].sort_values(by=['fips'])
age_df.head()

,STNAME,CTYNAME,fips,AGEGRP,TOT_POP,pct_male
0,Alabama,Autauga County,1001.0,0,54571,48.687032
190,Alabama,Baldwin County,1003.0,0,182265,48.937536
380,Alabama,Barbour County,1005.0,0,27457,53.086645
570,Alabama,Bibb County,1007.0,0,22915,53.680995
760,Alabama,Blount County,1009.0,0,57322,49.478385


In [34]:
for i in range(1, 19):
    agegrp = male_df[male_df["AGEGRP"] == i][["fips", "AGEGRP", "TOT_POP" ]].sort_values(by=['fips'])
    age_df[Age_Mapping[i]] = np.array(agegrp["TOT_POP"]) / np.array(age_df["TOT_POP"])

In [36]:
age_df.head()

,STNAME,CTYNAME,fips,AGEGRP,TOT_POP,pct_male,2010:AgeGroup01:0-4,2010:AgeGroup02:5-9,2010:AgeGroup03:10-14,2010:AgeGroup04:15-19,...,2010:AgeGroup09:40-44,2010:AgeGroup10:45-49,2010:AgeGroup11:50-54,2010:AgeGroup12:55-59,2010:AgeGroup13:60-64,2010:AgeGroup14:65-69,2010:AgeGroup15:70-74,2010:AgeGroup16:75-79,2010:AgeGroup17:80-84,2010:AgeGroup18:85+
0,Alabama,Autauga County,1001.0,0,54571,48.687032,0.065584,0.073134,0.078613,0.078613,...,0.074875,0.079383,0.070972,0.056495,0.050888,0.041725,0.031812,0.022924,0.013395,0.010097
190,Alabama,Baldwin County,1003.0,0,182265,48.937536,0.061219,0.063638,0.065432,0.063644,...,0.065811,0.073689,0.074013,0.068708,0.065904,0.055820,0.041857,0.030714,0.021584,0.017738
380,Alabama,Barbour County,1005.0,0,27457,53.086645,0.061988,0.059803,0.058237,0.063044,...,0.065885,0.076775,0.069563,0.066176,0.065521,0.046983,0.034527,0.024948,0.019776,0.016134
570,Alabama,Bibb County,1007.0,0,22915,53.680995,0.060135,0.061314,0.062841,0.067336,...,0.071351,0.078769,0.068994,0.061139,0.058215,0.042156,0.033035,0.023958,0.015492,0.012175
760,Alabama,Blount County,1009.0,0,57322,49.478385,0.063082,0.067688,0.071247,0.070357,...,0.070357,0.072625,0.068421,0.064443,0.062489,0.050452,0.038153,0.026465,0.018056,0.014096


In [55]:
#check to make sure ages sum to 1
age_df['Sum_of_ages'] = age_df[['2010:AgeGroup01:0-4', '2010:AgeGroup02:5-9', '2010:AgeGroup03:10-14',
       '2010:AgeGroup04:15-19', '2010:AgeGroup05:20-24',
       '2010:AgeGroup06:25-29', '2010:AgeGroup07:30-34',
       '2010:AgeGroup08:35-39', '2010:AgeGroup09:40-44',
       '2010:AgeGroup10:45-49', '2010:AgeGroup11:50-54',
       '2010:AgeGroup12:55-59', '2010:AgeGroup13:60-64',
       '2010:AgeGroup14:65-69', '2010:AgeGroup15:70-74',
       '2010:AgeGroup16:75-79', '2010:AgeGroup17:80-84',
       '2010:AgeGroup18:85+']].sum(axis = 1)
age_df[(abs(age_df["Sum_of_ages"]) > 1.0001)]  #ok!

,STNAME,CTYNAME,fips,AGEGRP,TOT_POP,pct_male,2010:AgeGroup01:0-4,2010:AgeGroup02:5-9,2010:AgeGroup03:10-14,2010:AgeGroup04:15-19,...,2010:AgeGroup10:45-49,2010:AgeGroup11:50-54,2010:AgeGroup12:55-59,2010:AgeGroup13:60-64,2010:AgeGroup14:65-69,2010:AgeGroup15:70-74,2010:AgeGroup16:75-79,2010:AgeGroup17:80-84,2010:AgeGroup18:85+,Sum_of_ages


## Step 4: Adding Race/Ethnicity Percentages (Adding Together Male/Female)

In [112]:
r_e_df = newdf[newdf['AGEGRP'] == 0]
r_e_df = r_e_df[['fips', 'CTYNAME', 'TOT_POP', 'WA_MALE', 'WA_FEMALE', 'BA_MALE',
       'BA_FEMALE', 'IA_MALE', 'IA_FEMALE', 'AA_MALE', 'AA_FEMALE', 'NA_MALE',
       'NA_FEMALE', 'WAC_MALE', 'WAC_FEMALE',
       'BAC_MALE', 'BAC_FEMALE', 'IAC_MALE', 'IAC_FEMALE', 'AAC_MALE',
       'AAC_FEMALE', 'NAC_MALE', 'NAC_FEMALE', 'H_MALE',
       'H_FEMALE']]

In [113]:
r_e_df.head()

,fips,CTYNAME,TOT_POP,WA_MALE,WA_FEMALE,BA_MALE,BA_FEMALE,IA_MALE,IA_FEMALE,AA_MALE,...,BAC_MALE,BAC_FEMALE,IAC_MALE,IAC_FEMALE,AAC_MALE,AAC_FEMALE,NAC_MALE,NAC_FEMALE,H_MALE,H_FEMALE
0,1001.0,Autauga County,54571,21295,22002,4559,5130,119,139,200,...,4704,5306,277,314,300,409,42,37,694,616
190,1003.0,Baldwin County,182265,78334,81376,8396,8878,662,677,540,...,8804,9327,1295,1346,783,1072,127,84,4425,3567
380,1005.0,Barbour County,27457,7463,6524,6808,6133,93,73,59,...,6898,6182,162,116,67,57,33,23,777,610
570,1007.0,Bibb County,22915,9055,8494,3079,1981,35,33,13,...,3119,2028,84,76,26,30,33,9,298,108
760,1009.0,Blount County,57322,27398,27970,420,397,178,173,45,...,493,472,364,376,81,134,45,31,2569,2057


In [114]:
r_e_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3142 entries, 0 to 596790
Data columns (total 25 columns):
fips          3141 non-null float64
CTYNAME       3142 non-null object
TOT_POP       3142 non-null int64
WA_MALE       3142 non-null int64
WA_FEMALE     3142 non-null int64
BA_MALE       3142 non-null int64
BA_FEMALE     3142 non-null int64
IA_MALE       3142 non-null int64
IA_FEMALE     3142 non-null int64
AA_MALE       3142 non-null int64
AA_FEMALE     3142 non-null int64
NA_MALE       3142 non-null int64
NA_FEMALE     3142 non-null int64
WAC_MALE      3142 non-null int64
WAC_FEMALE    3142 non-null int64
BAC_MALE      3142 non-null int64
BAC_FEMALE    3142 non-null int64
IAC_MALE      3142 non-null int64
IAC_FEMALE    3142 non-null int64
AAC_MALE      3142 non-null int64
AAC_FEMALE    3142 non-null int64
NAC_MALE      3142 non-null int64
NAC_FEMALE    3142 non-null int64
H_MALE        3142 non-null int64
H_FEMALE      3142 non-null int64
dtypes: float64(1), int64(23), object(

In [115]:
r_e_df['CEN:2010:WA'] = (r_e_df['WA_MALE'] + r_e_df['WA_FEMALE']) / r_e_df["TOT_POP"]
r_e_df['CEN:2010:BA'] = (r_e_df['BA_MALE'] + r_e_df['BA_FEMALE']) / r_e_df["TOT_POP"]
r_e_df['CEN:2010:IA'] = (r_e_df['IA_MALE'] + r_e_df['IA_FEMALE']) / r_e_df["TOT_POP"]
r_e_df['CEN:2010:AA'] = (r_e_df['AA_MALE'] + r_e_df['AA_FEMALE']) / r_e_df["TOT_POP"]
r_e_df['CEN:2010:NA'] = (r_e_df['NA_MALE'] + r_e_df['NA_FEMALE']) / r_e_df["TOT_POP"]
r_e_df['CEN:2010:WAC'] = (r_e_df['WAC_MALE'] + r_e_df['WAC_FEMALE']) / r_e_df["TOT_POP"]
r_e_df['CEN:2010:BAC'] = (r_e_df['BAC_MALE'] + r_e_df['BAC_FEMALE']) / r_e_df["TOT_POP"]
r_e_df['CEN:2010:IAC'] = (r_e_df['IAC_MALE'] + r_e_df['IAC_FEMALE']) / r_e_df["TOT_POP"]
r_e_df['CEN:2010:AAC'] = (r_e_df['AAC_MALE'] + r_e_df['AAC_FEMALE']) / r_e_df["TOT_POP"]
r_e_df['CEN:2010:NAC'] = (r_e_df['NAC_MALE'] + r_e_df['NAC_FEMALE']) / r_e_df["TOT_POP"]
r_e_df['CEN:2010:H'] = (r_e_df['H_MALE'] + r_e_df['H_FEMALE']) / r_e_df["TOT_POP"]

In [116]:
r_e_df.head().iloc[0]

fips                      1001
CTYNAME         Autauga County
TOT_POP                  54571
WA_MALE                  21295
WA_FEMALE                22002
BA_MALE                   4559
BA_FEMALE                 5130
IA_MALE                    119
IA_FEMALE                  139
AA_MALE                    200
AA_FEMALE                  284
NA_MALE                     29
NA_FEMALE                   18
WAC_MALE                 21633
WAC_FEMALE               22391
BAC_MALE                  4704
BAC_FEMALE                5306
IAC_MALE                   277
IAC_FEMALE                 314
AAC_MALE                   300
AAC_FEMALE                 409
NAC_MALE                    42
NAC_FEMALE                  37
H_MALE                     694
H_FEMALE                   616
CEN:2010:WA           0.793407
CEN:2010:BA           0.177549
CEN:2010:IA         0.00472779
CEN:2010:AA         0.00886918
CEN:2010:NA        0.000861263
CEN:2010:WAC          0.806729
CEN:2010:BAC          0.183431
CEN:2010

In [124]:
r_e_df.tail()

,fips,CTYNAME,TOT_POP,WA_MALE,WA_FEMALE,BA_MALE,BA_FEMALE,IA_MALE,IA_FEMALE,AA_MALE,...,CEN:2010:BA,CEN:2010:IA,CEN:2010:AA,CEN:2010:NA,CEN:2010:WAC,CEN:2010:BAC,CEN:2010:IAC,CEN:2010:AAC,CEN:2010:NAC,CEN:2010:H
596030,56037.0,Sweetwater County,43806,21788,19951,284,205,260,246,149,...,0.011163,0.011551,0.008081,0.001278,0.967014,0.014975,0.019221,0.011825,0.002739,0.152696
596220,56039.0,Teton County,21294,10791,9709,41,20,110,94,104,...,0.002865,0.009580,0.011365,0.001268,0.974312,0.004790,0.014699,0.016765,0.002818,0.149854
596410,56041.0,Uinta County,21118,10303,10082,34,29,115,123,22,...,0.002983,0.011270,0.003125,0.001799,0.980064,0.005540,0.020362,0.007103,0.003457,0.087840
596600,56043.0,Washakie County,8533,4079,4096,13,11,70,65,26,...,0.002813,0.015821,0.006914,0.000938,0.973397,0.007032,0.025665,0.008086,0.001641,0.136177
596790,56045.0,Weston County,7208,3649,3305,15,7,58,39,5,...,0.003052,0.013457,0.002913,0.000416,0.979883,0.005827,0.024834,0.004578,0.000555,0.029967


In [119]:
age_df.tail()

,STNAME,CTYNAME,fips,AGEGRP,TOT_POP,pct_male,2010:AgeGroup01:0-4,2010:AgeGroup02:5-9,2010:AgeGroup03:10-14,2010:AgeGroup04:15-19,...,2010:AgeGroup10:45-49,2010:AgeGroup11:50-54,2010:AgeGroup12:55-59,2010:AgeGroup13:60-64,2010:AgeGroup14:65-69,2010:AgeGroup15:70-74,2010:AgeGroup16:75-79,2010:AgeGroup17:80-84,2010:AgeGroup18:85+,Sum_of_ages
596220,Wyoming,Teton County,56039.0,0,21294,52.554710,0.060252,0.051188,0.049263,0.045271,...,0.071147,0.075092,0.072133,0.058467,0.040105,0.023340,0.017000,0.009580,0.008500,1.0
596410,Wyoming,Uinta County,56041.0,0,21118,50.487736,0.084336,0.084667,0.083152,0.074297,...,0.067857,0.083436,0.072213,0.049200,0.033100,0.020030,0.015674,0.010844,0.009092,1.0
596600,Wyoming,Washakie County,56043.0,0,8533,49.865229,0.070315,0.071839,0.068323,0.064573,...,0.069846,0.077112,0.074065,0.065159,0.054494,0.041134,0.033165,0.026017,0.021915,1.0
596790,Wyoming,Weston County,56045.0,0,7208,52.580466,0.056881,0.065344,0.058962,0.056188,...,0.077414,0.085322,0.083518,0.063402,0.044950,0.040233,0.026360,0.022475,0.025388,1.0
16720,Alaska,Prince of Wales-Hyder Census Area,NaN,0,5559,55.225760,0.074834,0.065120,0.074834,0.062601,...,0.084727,0.087786,0.093902,0.065300,0.047311,0.024645,0.016550,0.007555,0.004497,1.0


In [134]:
r_e_df.loc[16720][["fips", "CTYNAME", "CEN:2010:WA", "CEN:2010:BA" ]]   #make sure this lines up propoerly on indexes

fips                                         NaN
CTYNAME        Prince of Wales-Hyder Census Area
CEN:2010:WA                             0.507285
CEN:2010:BA                           0.00323799
Name: 16720, dtype: object

In [126]:
print ("age df shape: ",age_df.shape )
print ("r_e_df shape: ",r_e_df.shape )

age df shape:  (3142, 25)
r_e_df shape:  (3142, 36)


In [132]:
finaldf = r_e_df.merge(age_df, how = "outer")
finaldf.shape

(3142, 58)

In [139]:
#checking that it properly matched on indices and not on order (i.e. that Alaska is right, since it was out of order)
finaldf.tail()[["CTYNAME", "CEN:2010:WA", "CEN:2010:BA"]]  

,CTYNAME,CEN:2010:WA,CEN:2010:BA
3137,Sweetwater County,0.952815,0.011163
3138,Teton County,0.962713,0.002865
3139,Uinta County,0.965290,0.002983
3140,Washakie County,0.958045,0.002813
3141,Weston County,0.964761,0.003052


In [140]:
finaldf[finaldf["CTYNAME"] == "Prince of Wales-Hyder Census Area"][["CTYNAME", "CEN:2010:WA", "CEN:2010:BA"]]  

,CTYNAME,CEN:2010:WA,CEN:2010:BA
88,Prince of Wales-Hyder Census Area,0.507285,0.003238


In [141]:
finaldf.to_csv('../data/Census_2010_rates_2.csv')

# Old / Exploring Original DataSet

In [ ]:
#Old: First pass with just age and % male

In [171]:
first_pass_final.to_csv('../data/Census_2010_rates_1.csv')

In [5]:
## Exploring Race/Ethnicity

In [6]:
df.columns

Index(['SUMLEV', 'STATE', 'COUNTY', 'STNAME', 'CTYNAME', 'YEAR', 'AGEGRP',
       'TOT_POP', 'TOT_MALE', 'TOT_FEMALE', 'WA_MALE', 'WA_FEMALE', 'BA_MALE',
       'BA_FEMALE', 'IA_MALE', 'IA_FEMALE', 'AA_MALE', 'AA_FEMALE', 'NA_MALE',
       'NA_FEMALE', 'TOM_MALE', 'TOM_FEMALE', 'WAC_MALE', 'WAC_FEMALE',
       'BAC_MALE', 'BAC_FEMALE', 'IAC_MALE', 'IAC_FEMALE', 'AAC_MALE',
       'AAC_FEMALE', 'NAC_MALE', 'NAC_FEMALE', 'NH_MALE', 'NH_FEMALE',
       'NHWA_MALE', 'NHWA_FEMALE', 'NHBA_MALE', 'NHBA_FEMALE', 'NHIA_MALE',
       'NHIA_FEMALE', 'NHAA_MALE', 'NHAA_FEMALE', 'NHNA_MALE', 'NHNA_FEMALE',
       'NHTOM_MALE', 'NHTOM_FEMALE', 'NHWAC_MALE', 'NHWAC_FEMALE',
       'NHBAC_MALE', 'NHBAC_FEMALE', 'NHIAC_MALE', 'NHIAC_FEMALE',
       'NHAAC_MALE', 'NHAAC_FEMALE', 'NHNAC_MALE', 'NHNAC_FEMALE', 'H_MALE',
       'H_FEMALE', 'HWA_MALE', 'HWA_FEMALE', 'HBA_MALE', 'HBA_FEMALE',
       'HIA_MALE', 'HIA_FEMALE', 'HAA_MALE', 'HAA_FEMALE', 'HNA_MALE',
       'HNA_FEMALE', 'HTOM_MALE', 'HTOM_FEMALE

In [15]:
df_2010_totalpop = df[df['YEAR'] == 1][df[df['YEAR'] == 1]['AGEGRP'] == 0]
df_2010_totalpop.head()

,SUMLEV,STATE,COUNTY,STNAME,CTYNAME,YEAR,AGEGRP,TOT_POP,TOT_MALE,TOT_FEMALE,...,HWAC_MALE,HWAC_FEMALE,HBAC_MALE,HBAC_FEMALE,HIAC_MALE,HIAC_FEMALE,HAAC_MALE,HAAC_FEMALE,HNAC_MALE,HNAC_FEMALE
0,50,1,1,Alabama,Autauga County,1,0,54571,26569,28002,...,607,538,57,48,26,32,9,11,19,10
190,50,1,3,Alabama,Baldwin County,1,0,182265,89196,93069,...,4153,3325,156,154,189,163,54,48,45,26
380,50,1,5,Alabama,Barbour County,1,0,27457,14576,12881,...,637,509,66,49,69,44,1,0,14,10
570,50,1,7,Alabama,Bibb County,1,0,22915,12301,10614,...,257,87,19,18,1,3,5,1,23,5
760,50,1,9,Alabama,Blount County,1,0,57322,28362,28960,...,2442,1978,60,38,66,35,2,6,28,18


In [11]:
df_2010_totalpop = df_2010_totalpop[['STNAME', 'CTYNAME', 'YEAR', 'TOT_POP', 'TOT_MALE', 'TOT_FEMALE', 'WA_MALE', 'WA_FEMALE', 'BA_MALE',
       'BA_FEMALE', 'IA_MALE', 'IA_FEMALE', 'AA_MALE', 'AA_FEMALE', 'NA_MALE',
       'NA_FEMALE', 'TOM_MALE', 'TOM_FEMALE', 'WAC_MALE', 'WAC_FEMALE',
       'BAC_MALE', 'BAC_FEMALE', 'IAC_MALE', 'IAC_FEMALE', 'AAC_MALE',
       'AAC_FEMALE', 'NAC_MALE', 'NAC_FEMALE', 'H_MALE',
       'H_FEMALE' ]]

In [12]:
df_2010_totalpop

,STNAME,CTYNAME,YEAR,TOT_POP,TOT_MALE,TOT_FEMALE,WA_MALE,WA_FEMALE,BA_MALE,BA_FEMALE,...,BAC_MALE,BAC_FEMALE,IAC_MALE,IAC_FEMALE,AAC_MALE,AAC_FEMALE,NAC_MALE,NAC_FEMALE,H_MALE,H_FEMALE
0,Alabama,Autauga County,1,54571,26569,28002,21295,22002,4559,5130,...,4704,5306,277,314,300,409,42,37,694,616
190,Alabama,Baldwin County,1,182265,89196,93069,78334,81376,8396,8878,...,8804,9327,1295,1346,783,1072,127,84,4425,3567
380,Alabama,Barbour County,1,27457,14576,12881,7463,6524,6808,6133,...,6898,6182,162,116,67,57,33,23,777,610
570,Alabama,Bibb County,1,22915,12301,10614,9055,8494,3079,1981,...,3119,2028,84,76,26,30,33,9,298,108
760,Alabama,Blount County,1,57322,28362,28960,27398,27970,420,397,...,493,472,364,376,81,134,45,31,2569,2057
950,Alabama,Bullock County,1,10914,5912,5002,1761,1107,3999,3760,...,4031,3782,62,76,26,15,47,37,439,338
1140,Alabama,Butler County,1,20947,9838,11109,5498,5944,4141,4969,...,4184,5007,54,65,109,97,6,7,106,85
1330,Alabama,Calhoun County,1,118572,57176,61396,44186,46430,11367,13180,...,11803,13667,655,628,550,714,118,122,2122,1771
1520,Alabama,Chambers County,1,34215,16363,17852,9940,10356,6125,7200,...,6212,7309,91,98,122,102,14,8,331,205
1710,Alabama,Cherokee County,1,25989,12888,13101,11982,12249,618,592,...,677,644,199,161,29,48,13,7,157,163


In [17]:
df_2010_totalpop[['WA_MALE', "WAC_MALE" ]].head()

,WA_MALE,WAC_MALE
0,21295,21633
190,78334,79452
380,7463,7546
570,9055,9133
760,27398,27670


In [28]:
df_2010_totalpop['W_M_comp'] = df_2010_totalpop['WAC_MALE'] / df_2010_totalpop["WA_MALE" ]

In [29]:
df_2010_totalpop[['W_M_comp']].describe()

,W_M_comp
count,3142.000000
mean,1.020159
std,0.031556
min,1.000000
25%,1.010346
50%,1.014538
75%,1.020789
max,1.668305


In [30]:
df_2010_totalpop['B_M_comp'] = df_2010_totalpop['BAC_MALE'] / df_2010_totalpop["BA_MALE" ]

In [31]:
df_2010_totalpop[['B_M_comp']]

,B_M_comp
0,1.031805
190,1.048595
380,1.013220
570,1.012991
760,1.173810
950,1.008002
1140,1.010384
1330,1.038357
1520,1.014204
1710,1.095469


In [32]:
df_2010_totalpop[['B_M_comp']].describe()

,B_M_comp
count,3123.000000
mean,inf
std,NaN
min,1.000000
25%,1.065105
50%,1.210809
75%,1.498538
max,inf


In [33]:
#prett6y big difference, at least in black & african american communities which i checked. SO...i think I should pull in both, and 
# then see which one gives me a better prediction

In [196]:
### Old: Double checked: does it all add up for 2010? --> YEs!

In [197]:
ages_total = first_pass[first_pass["AGEGRP"] != 0]
ages_total = ages_total.groupby("fips").sum()["TOT_POP"]

In [198]:
ages_total = ages_total.reset_index()
ages_total.head(7)

,fips,TOT_POP
0,1001.0,54571
1,1003.0,182265
2,1005.0,27457
3,1007.0,22915
4,1009.0,57322
5,1011.0,10914
6,1013.0,20947


In [199]:
ages_total_given = first_pass[first_pass["AGEGRP"] == 0][['fips','TOT_POP']]

In [200]:
ages_total_given = ages_total_given.sort_values(by=['fips']).reset_index()
ages_total_given.head(7)

,index,fips,TOT_POP
0,0,1001.0,54571
1,190,1003.0,182265
2,380,1005.0,27457
3,570,1007.0,22915
4,760,1009.0,57322
5,950,1011.0,10914
6,1140,1013.0,20947


In [201]:
print(ages_total.shape)
print(ages_total_given.shape)

(3141, 2)
(3142, 3)


In [202]:
##  doesn't matter
##(ages_total['TOT_POP'] == ages_total_given['TOT_POP']).sum()

In [203]:
#confirmed! Ages are exhaustive